# Working with Classical Optimization Solvers

We can use classical optimization solvers (CPLEX and Gurobi) with Qiskit Optimization.
Docplex and Gurobipy are the Python APIs for CPLEX and Gurobi, respectively.
We can load and save an optimization model by Docplex and Gurobipy and can apply CPLEX and Gurobi to `QuadraticProgram`.

If you want to use the solver of CPLEX, you need to install `pip install 'qiskit-optimization[cplex]'`. Docplex is supposed to be installed with Qiskit Optimization.

If you want to use Gurobi and Gurobipy, you need to install `pip install 'qiskit-optimization[gurobi]'`.

Note that these solvers you install via pip are free versions, which have some limitations such as number of variables. See the following pages for details.
- https://pypi.org/project/cplex/
- https://pypi.org/project/gurobipy/

## Translators between `QuadraticProgram` and Docplex/Gurobipy

First, we define an optimization problem by Docplex and Gurobipy.

In [1]:
# docplex model
from docplex.mp.model import Model
docplex_model = Model('docplex')
x = docplex_model.binary_var('x')
y = docplex_model.integer_var(-1, 4, 'y')
docplex_model.maximize(x * y)
docplex_model.add_constraint(x <= y)
print(docplex_model.export_as_lp_string())
print(docplex_model.solve())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex

Maximize
 obj: [ 2 x*y ]/2
Subject To
 c1: x - y <= 0

Bounds
 0 <= x <= 1
 -1 <= y <= 4

Binaries
 x

Generals
 y
End

solution for: docplex
objective: 4
x=1
y=4



In [2]:
# gurobi model
import gurobipy as gp
gurobipy_model = gp.Model('gurobi')
x = gurobipy_model.addVar(vtype=gp.GRB.BINARY, name="x")
y = gurobipy_model.addVar(vtype=gp.GRB.INTEGER, lb=-1, ub=4, name="y")
gurobipy_model.setObjective(x * y, gp.GRB.MAXIMIZE)
gurobipy_model.addConstr(x - y <= 0)
gurobipy_model.Params.OutputFlag = 0
gurobipy_model.optimize()
gurobipy_model.display()
print('objective:', gurobipy_model.ObjVal)
print('x:', gurobipy_model.X)

Restricted license - for non-production use only - expires 2022-01-13
Maximize
   <gurobi.QuadExpr: 0.0 + [ x * y ]>
Subject To
   R0 : <gurobi.LinExpr: x + -1.0 y> <= 0.0
Bounds
   -1.0 <= y <= 4.0
Binaries
   ['x']
General Integers
   ['y']
objective: 4.0
x: [1.0, 4.0]


We can generate `QuadraticProgram` object from both Docplex and Gurobipy models. We see that the two `QuadraticProgram` objects generated from Docplex and Gurobipy are identical.

In [3]:
from qiskit_optimization.translators import from_docplex_mp, from_gurobipy
qp = from_docplex_mp(docplex_model)
print('from docpblex')
print(qp.export_as_lp_string())
print('-------------')
print('from gurobipy')
qp2 = from_gurobipy(gurobipy_model)
print(qp2.export_as_lp_string())

from docpblex
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex

Maximize
 obj: [ 2 x*y ]/2
Subject To
 c0: x - y <= 0

Bounds
 0 <= x <= 1
 -1 <= y <= 4

Binaries
 x

Generals
 y
End

-------------
from gurobipy
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: gurobi

Maximize
 obj: [ 2 x*y ]/2
Subject To
 R0: x - y <= 0

Bounds
 0 <= x <= 1
 -1 <= y <= 4

Binaries
 x

Generals
 y
End



We can generate a Docplex model and a Gurobipy model from `QuadraticProgram` too.

In [4]:
from qiskit_optimization.translators import to_gurobipy, to_docplex_mp
gmod = to_gurobipy(from_docplex_mp(docplex_model))
print('convert docplex to gurobipy')
gmod.display()

dmod = to_docplex_mp(from_gurobipy(gurobipy_model))
print('\nconvert gurobi to docplex')
dmod.prettyprint()

convert docplex to gurobipy
Maximize
   <gurobi.QuadExpr: 0.0 + [ x * y ]>
Subject To
   c0 : <gurobi.LinExpr: x + -1.0 y> <= 0.0
Bounds
   -1.0 <= y <= 4.0
Binaries
   ['x']
General Integers
   ['y']

convert gurobi to docplex
// This file has been generated by DOcplex
// model name is: gurobi
// single vars section
dvar bool x;
dvar int y;

maximize
 [ x*y ];
 
subject to {
 R0:
  x - y <= 0;

}


### Indicator constraints of Docplex

`from_docplex_mp` supports indicator constraints, e.g., `u = 0 => x + y <= z` (u: binary variable) when we convert a Docplex model into `QuadraticProgram`. It converts indicator constraints into linear constraints using the big-M formulation.

In [5]:
ind_mod = Model('docplex')
x = ind_mod.binary_var('x')
y = ind_mod.integer_var(-1, 2, 'y')
z = ind_mod.integer_var(-1, 2, 'z')
ind_mod.maximize(3 * x + y - z)
ind_mod.add_indicator(x, y >= z, 1)
print(ind_mod.export_as_lp_string())
print(ind_mod.solve())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex

Maximize
 obj: 3 x + y - z
Subject To
 lc1: x = 1 -> y - z >= 0

Bounds
 0 <= x <= 1
 -1 <= y <= 2
 -1 <= z <= 2

Binaries
 x

Generals
 y z
End

solution for: docplex
objective: 6
x=1
y=2
z=-1



We get the same solution by QAOA.

In [6]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA

qins = QuantumInstance(backend=Aer.get_backend('aer_simulator'), shots=1000)
meo = MinimumEigenOptimizer(QAOA(COBYLA(maxiter=100), quantum_instance=qins))
qp = from_docplex_mp(ind_mod)
result = meo.solve(qp)
print(result)
print()
for sample in result.samples[:5]:
    print(sample)

optimal function value: 6.0
optimal value: [ 1.  2. -1.]
status: SUCCESS

SolutionSample(x=array([ 1.,  2., -1.]), fval=6.0, probability=0.04599999999999999, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([ 1.,  1., -1.]), fval=5.0, probability=0.013, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 2., 0.]), fval=5.0, probability=0.026, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([ 1.,  0., -1.]), fval=4.0, probability=0.054000000000000006, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 1., 0.]), fval=4.0, probability=0.009999999999999998, status=<OptimizationResultStatus.SUCCESS: 0>)


## CplexSolver and GurobiSolver

Qiskit optimization supports the classical solvers of CPLEX and Gurobi as `CplexSolver` and `GurobiSolver`, respectively.
We can solve `QuadraticProgram` with `CplexSolver` and `GurobiSolver` as follows.

In [7]:
from qiskit_optimization.algorithms import CplexOptimizer, GurobiOptimizer
print(qp.export_as_lp_string())
print('cplex')
print(CplexOptimizer().solve(qp))
print()
print('gurobi')
print(GurobiOptimizer().solve(qp))

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex

Maximize
 obj: 3 x + y - z
Subject To
 c0: - 3 x + y - z >= -3

Bounds
 0 <= x <= 1
 -1 <= y <= 2
 -1 <= z <= 2

Binaries
 x

Generals
 y z
End

cplex
optimal function value: 6.0
optimal value: [ 1.  2. -1.]
status: SUCCESS

gurobi
optimal function value: 6.0
optimal value: [ 1.  2. -1.]
status: SUCCESS


We can set the solver parameter of CPLEX as follows. See [Parameters of CPLEX](https://www.ibm.com/docs/en/icos/20.1.0?topic=cplex-parameters) for details of the parameters.

In [8]:
CplexOptimizer(disp=True, cplex_parameters={'threads': 1, 'timelimit': 0.1}).solve(qp)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               0.10000000000000001
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Found incumbent of value 3.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 3 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.00 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.00 ticks)


optimal function value: 6.0
optimal value: [ 1.  2. -1.]
status: SUCCESS

In [9]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright